In [18]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
path = '../archive/anime_with_synopsis.csv'
df = pd.read_csv(path)

In [20]:
df.rename(columns={"sypnopsis" : "Synopsis"}, inplace=True)

In [21]:
df.sample(10)

,MAL_ID,Name,Score,Genres,Synopsis
11909,35476,Little Charo 4: Eigo de Aruku New York,Unknown,"Adventure, Kids",No synopsis information has been added to this...
10885,33689,Dive to Blue,6.08,Music,"music video for the song ""Dive to Blue"" by Umi..."
1375,1577,Taiho Shichau zo,7.46,"Action, Comedy, Police, Seinen",unning late on her first day as a patrol woman...
8644,27915,Captain,Unknown,"Sports, Shounen",Before Captain became a TV series it was a one...
5916,12451,Gattai Robot Atranger,Unknown,Mecha,langer is a plastic model series first release...
9656,30928,Tobidashi wa Abunaizo! Mushi Mushi Mura no Kou...,Unknown,"Drama, Kids",ducational film about traffic safety starring ...
3734,5921,Utsunomiko,Unknown,"Adventure, Demons, Fantasy",In the chaos of the Jinshin-no-Ran civil war o...
15302,41340,Uchida Shungicu no Noroi no One-Piece,5.94,"Horror, Romance, Shoujo",omnibus horror TV special where three girls en...
12474,36506,Fuwafuwa Hour: Pui Pui & Muu Muu,Unknown,"Kids, Fantasy","Fairies living in a fluffy forest, where both ..."
112,133,Green Green,6.21,"Comedy, Ecchi, Romance, School, Slice of Life",Kanenone Gakuen is an all-male boarding school...


In [22]:
df['Synopsis'] = df['Synopsis'].fillna('')

In [23]:
docs = []
for i in range(len(df)):
    docs.append(df['Synopsis'][i])


In [24]:
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(docs)
count_df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names_out())
count_df.sample(10)

,00,000,000th,001,0015,002,003,0060,0068,007,...,青青草原,飯野真澄,高河,鳥羽和一,鶺鴒余暇,鶺鴒診断,麻城ゆう,黄色いしあわせ,박사,카프
9631,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8779,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7140,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
929,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10577,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2650,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2918,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\35191\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Cleaning up and tokenizing text lmao xd

In [26]:
from nltk.tokenize import word_tokenize
def tokenize(text):
    words = word_tokenize(text)
    words = [w.lower() for w in words]
    words = [w for w in words if w not in stopwords.words('english')]
    return words
    

### Building the inverted index

In [32]:
def build_inverted_index(words):
    inverted = {}
    for index, word in enumerate(words):
        locations = inverted.setdefault(word, [])
        locations.append(index)
    return inverted

def inverted_index_add(inverted, doc_id, doc_index):
    for word in doc_index.keys():
        locations = doc_index[word]
        indices = inverted.setdefault(word, {})
        indices[doc_id] = locations
    return inverted

In [28]:
df.head()

,MAL_ID,Name,Score,Genres,Synopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [35]:
import tqdm

In [37]:
inverted_doc_indexes = {}
files_with_index = []
files_with_tokens = {}

doc_id = 0

for anime in df['Name']:
    words = tokenize(df['Synopsis'][doc_id])
    files_with_tokens[doc_id] = words
    doc_index = build_inverted_index(words)
    inverted_index_add(inverted_doc_indexes, doc_id, doc_index)
    files_with_index.append(doc_id)
    doc_id += 1


In [46]:
def boolean_search(inverted, animes, query):
    query_words = tokenize(query)
    query_words = [w for w in query_words if w in inverted.keys()]
    if len(query_words) == 0:
        return []
    result = inverted[query_words[0]]
    for word in query_words[1:]:
        result = result.intersection(inverted[word])
    return [animes[doc_id] for doc_id in result]

{11: [2, 17],
 521: [26, 38, 59, 67, 88, 118],
 3624: [75],
 4789: [18],
 12299: [2]}